---
# *Linear Regression*
---

 1. ***Linear Regression : 선형 회귀***
 2. ***Autograd : 자동 미분***
 3. ***Multivariable Linear regression : 다중 선형 회귀***
 4. ***nn.Module***
 5. ***class***
 6. ***Mini Batch and Data Load***
 7. ***Custom Dataset***

In [63]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더
from torch.utils.data import Dataset
torch.manual_seed(1)

## ***1. Linear Regression***

> requires_grad=True : 자동 미분 기능 적용, 즉 텐서에 대한 기울기 저장 <br> optimizer.zero_grad() : 누적되는 값을 초기화 <br> X.backward() : 해당(X)수식을 미분 <br> optimizer.step() <br> X.item()

In [2]:
# dataset
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])
print(x_train.shape)
print(y_train.shape)

# 가중치 초기화
W = torch.zeros(1, requires_grad=True) 
b = torch.zeros(1, requires_grad=True)

# 가설 세우기
hypothesis = x_train * W + b
print(hypothesis)

# 비용 함수 선언 : cost(W,b)=1/n∑([y(i)−H(x(i))]**2)
cost = torch.mean((hypothesis - y_train) ** 2) # mse
print(cost)

# 경사하강법 구현
optimizer = optim.SGD([W, b], lr=0.01)

epochs = 2001
for i in range(epochs):
    # 가설설정
    hypothesis = x_train * W + b
    #비용함수 선언
    cost = torch.mean((hypothesis - y_train) ** 2)
    # gradient를 0으로 초기화
    optimizer.zero_grad() 
    # 비용 함수를 미분하여 gradient 계산
    cost.backward() 
    # W와 b를 업데이트
    optimizer.step() 
    
    # 100번마다 로그 출력
    if i % 500 == 0:
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            i, epochs, W.item(), b.item(), cost.item()))

torch.Size([3, 1])
torch.Size([3, 1])
tensor([[0.],
        [0.],
        [0.]], grad_fn=<AddBackward0>)
tensor(18.6667, grad_fn=<MeanBackward0>)
Epoch    0/2001 W: 0.187, b: 0.080 Cost: 18.666666
Epoch  500/2001 W: 1.903, b: 0.221 Cost: 0.007024
Epoch 1000/2001 W: 1.971, b: 0.066 Cost: 0.000633
Epoch 1500/2001 W: 1.991, b: 0.020 Cost: 0.000057
Epoch 2000/2001 W: 1.997, b: 0.006 Cost: 0.000005


## ***2. Autograd***

 - *파이토치에서는 미분 계산을 자동화하여 경사 하강법을 손쉽게 사용할 수 있게 함*

In [3]:
# requires_grad=True : 이 텐서에 대한 기울기를 저장
# w.grad에 w의 미분값이 저장됨
w = torch.tensor(2.0, requires_grad=True)
print(w)

y = w**2 # 4
z = 2*y + 5 # 13
print(w, y, z)

z.backward()
print(z)

print('수식을 w로 미분한 값 : {}'.format(w.grad))

tensor(2., requires_grad=True)
tensor(2., requires_grad=True) tensor(4., grad_fn=<PowBackward0>) tensor(13., grad_fn=<AddBackward0>)
tensor(13., grad_fn=<AddBackward0>)
수식을 w로 미분한 값 : 8.0


## ***3. Multivariable Linear regression***

 1. ***다중 선형 회귀 구현***
 2. ***행렬 연산을 고려하여 다중 선형 회귀 구현***



##### ***`1. 다중 선형 회귀 구현`***


 - *최적의 학습률 : 1e-5*
     - *더 작게 설정할 시 너무 느린 학습률 / 더 크게 설정할 시 결측치 발생*
     - *단순 선형 회귀에서처럼 0.01로 학습률을 지정하지 않는 이유는 기울기 발산 때문 : 이유 더 알아보기*
     

In [13]:
# 훈련 데이터
x1_train = torch.FloatTensor([[73], [93], [89], [96], [73]])
x2_train = torch.FloatTensor([[80], [88], [91], [98], [66]])
x3_train = torch.FloatTensor([[75], [93], [90], [100], [70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# 가중치 w와 편향 b 초기화
w1 = torch.zeros(1, requires_grad=True)
w2 = torch.zeros(1, requires_grad=True)
w3 = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 구현
optimizer = optim.SGD([w1, w2, w3, b], lr=1e-5)

epochs = 5000
for i in range(epochs + 1):
    
    hypothesis = x1_train * w1 + x2_train * w2 + x3_train * w3 + b
    
    cost = torch.mean((hypothesis - y_train) ** 2)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if i % 500 == 0:
        print('Epoch {:4d}/{} w1: {:.3f} w2: {:.3f} w3: {:.3f} b: {:.3f} Cost: {:.6f}'.format(
            i, epochs, w1.item(), w2.item(), w3.item(), b.item(), cost.item()))

Epoch    0/5000 w1: 0.294 w2: 0.294 w3: 0.297 b: 0.003 Cost: 29661.800781
Epoch  500/5000 w1: 0.694 w2: 0.638 w3: 0.678 b: 0.009 Cost: 1.319511
Epoch 1000/5000 w1: 0.718 w2: 0.613 w3: 0.680 b: 0.009 Cost: 1.079378
Epoch 1500/5000 w1: 0.739 w2: 0.591 w3: 0.681 b: 0.010 Cost: 0.895453
Epoch 2000/5000 w1: 0.757 w2: 0.571 w3: 0.682 b: 0.011 Cost: 0.754389
Epoch 2500/5000 w1: 0.773 w2: 0.555 w3: 0.682 b: 0.011 Cost: 0.646035
Epoch 3000/5000 w1: 0.788 w2: 0.541 w3: 0.682 b: 0.012 Cost: 0.562648
Epoch 3500/5000 w1: 0.801 w2: 0.528 w3: 0.682 b: 0.012 Cost: 0.498317
Epoch 4000/5000 w1: 0.812 w2: 0.517 w3: 0.681 b: 0.013 Cost: 0.448554
Epoch 4500/5000 w1: 0.823 w2: 0.508 w3: 0.680 b: 0.014 Cost: 0.409898
Epoch 5000/5000 w1: 0.832 w2: 0.500 w3: 0.678 b: 0.014 Cost: 0.379734


##### ***`2. 행렬 연산을 고려하여 다중 선형 회귀 구현`***

> X.squeeze().detach()

In [16]:
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  80], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])
print('x shape : ', x_train.shape)
print('y shape : ', y_train.shape)

# 가중치와 편향 선언
W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
print('W shape : ', W.shape)
print('b shape : ', b.shape)

hypothesis = x_train.matmul(W) + b
print('hypothesis shape : ', hypothesis.shape)

x shape :  torch.Size([5, 3])
y shape :  torch.Size([5, 1])
W shape :  torch.Size([3, 1])
b shape :  torch.Size([1])
hypothesis shape :  torch.Size([5, 1])


In [18]:
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    # 편향 b는 브로드 캐스팅되어 각 샘플에 더해집니다.
    hypothesis = x_train.matmul(W) + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 5 == 0:
        print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
            epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
        ))

Epoch    0/20 hypothesis: tensor([154.0543, 185.1140, 175.7470, 198.6150, 141.2166]) Cost: 5.954494
Epoch    5/20 hypothesis: tensor([154.0545, 185.1137, 175.7535, 198.6133, 141.2178]) Cost: 5.941589
Epoch   10/20 hypothesis: tensor([154.0538, 185.1123, 175.7588, 198.6104, 141.2181]) Cost: 5.928727
Epoch   15/20 hypothesis: tensor([154.0530, 185.1109, 175.7641, 198.6075, 141.2184]) Cost: 5.915939
Epoch   20/20 hypothesis: tensor([154.0523, 185.1094, 175.7694, 198.6045, 141.2187]) Cost: 5.903165


## ***4. nn.Module***

 1. ***단순 선형 회귀 구현***
 2. ***다중 선형 회귀 구현***


##### ***`1. 단순 선형 회귀 구현`***

In [13]:
torch.manual_seed(1)

# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

# input_dim(x)=1, output_dim(y)=1 -> y = ax + b
model = nn.Linear(1,1)
print('model의 가중치 : {} \n model의 편향 : {}'.format(list(model.parameters())[0], list(model.parameters())[1]))
print('\n')

# optimizer 설정
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) 

epochs = 2000
for i in range(epochs+1):
    
    prediction = model(x_train)
    
    cost = F.mse_loss(prediction, y_train) # pytorch에서 제공하는 mse
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if i % 500 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
          i, epochs, cost.item()))
        
        
# test
new_var =  torch.FloatTensor([[4.0]]) 
pred_y = model(new_var) # forward 연산
# y = 2x 이므로 입력이 4라면 y가 8에 가까운 값이 나와야 제대로 학습이 된 것
print('\n')
print("훈련 후 입력이 4일 때의 예측값 :", pred_y) 
print('model의 가중치 : {} \n model의 편향 : {}'.format(list(model.parameters())[0], list(model.parameters())[1]))

model의 가중치 : Parameter containing:
tensor([[0.5153]], requires_grad=True) 
 model의 편향 : Parameter containing:
tensor([-0.4414], requires_grad=True)


Epoch    0/2000 Cost: 13.103541
Epoch  500/2000 Cost: 0.000407
Epoch 1000/2000 Cost: 0.000037
Epoch 1500/2000 Cost: 0.000003
Epoch 2000/2000 Cost: 0.000000


훈련 후 입력이 4일 때의 예측값 : tensor([[7.9989]], grad_fn=<AddmmBackward>)
model의 가중치 : Parameter containing:
tensor([[1.9994]], requires_grad=True) 
 model의 편향 : Parameter containing:
tensor([0.0014], requires_grad=True)


##### ***`2. 다중 선형 회귀 구현`***

In [43]:
# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# input_dim(x)=3, output_dim(y)=1 -> y = a1x1 + a2x2 + a3x3 + b
model = nn.Linear(3,1)
print(list(model.parameters()))

optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) 

epochs = 2000
for i in range(epochs + 1):
    
    predcition = model(x_train) #  model(x_train)은 model.forward(x_train)와 동일함.
    print(prediction)
    
    cost = F.mse_loss(prediction, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if i % 500 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
          i, epochs, cost.item()))
        

# test
new_var =  torch.FloatTensor([[73, 80, 75]]) 
pred_y = model(new_var) 
print("훈련 후 입력이 73, 80, 75일 때의 예측값 :", pred_y) 
print(list(model.parameters()))

[Parameter containing:
tensor([[ 0.0803, -0.0707,  0.1601]], requires_grad=True), Parameter containing:
tensor([0.0285], requires_grad=True)]
tensor([[153.9894],
        [182.8504],
        [181.2845],
        [197.4515],
        [138.9471]], grad_fn=<AddmmBackward>)


RuntimeError: Trying to backward through the graph a second time, but the saved intermediate results have already been freed. Specify retain_graph=True when calling backward the first time.

In [36]:
predcition = model(x_train) 
print(prediction)

tensor([[2.0008],
        [4.0002],
        [5.9995]], grad_fn=<AddmmBackward>)


In [37]:
model(x_train)

tensor([[ 83.6415],
        [ 98.1081],
        [ 97.9626],
        [106.7360],
        [ 74.0690]], grad_fn=<AddmmBackward>)

In [38]:
model

Linear(in_features=3, out_features=1, bias=True)

## ***5. Class***

 - *class 사용 시 왜 forward 함수를 호출하지 않고 바로 데이터를 넣을 수 있는지 의문*

In [39]:
torch.manual_seed(1)

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)
    
    
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)
        
    def forward(self, x):
        return self.linear(x)

In [41]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

model = LinearRegressionModel()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01) 

nb_epochs = 2000
for epoch in range(nb_epochs+1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.mse_loss(prediction, y_train) # <== 파이토치에서 제공하는 평균 제곱 오차 함수

    # cost로 H(x) 개선하는 부분
    # gradient를 0으로 초기화
    optimizer.zero_grad()
    # 비용 함수를 미분하여 gradient 계산
    cost.backward() # backward 연산
    # W와 b를 업데이트
    optimizer.step()

    if epoch % 500 == 0:
    # 100번마다 로그 출력
      print('Epoch {:4d}/{} Cost: {:.6f}'.format(
          epoch, nb_epochs, cost.item()
      ))

Epoch    0/2000 Cost: 18.562185
Epoch  500/2000 Cost: 0.018671
Epoch 1000/2000 Cost: 0.001682
Epoch 1500/2000 Cost: 0.000152
Epoch 2000/2000 Cost: 0.000014


In [42]:
# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

model = MultivariateLinearRegressionModel()

optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) 

nb_epochs = 2000
for epoch in range(nb_epochs+1):

    # H(x) 계산
    prediction = model(x_train)
    # model(x_train)은 model.forward(x_train)와 동일함.

    # cost 계산
    cost = F.mse_loss(prediction, y_train) # <== 파이토치에서 제공하는 평균 제곱 오차 함수

    # cost로 H(x) 개선하는 부분
    # gradient를 0으로 초기화
    optimizer.zero_grad()
    # 비용 함수를 미분하여 gradient 계산
    cost.backward()
    # W와 b를 업데이트
    optimizer.step()

    if epoch % 500 == 0:
    # 100번마다 로그 출력
      print('Epoch {:4d}/{} Cost: {:.6f}'.format(
          epoch, nb_epochs, cost.item()
      ))

Epoch    0/2000 Cost: 39633.414062
Epoch  500/2000 Cost: 9.315010
Epoch 1000/2000 Cost: 7.190376
Epoch 1500/2000 Cost: 5.568909
Epoch 2000/2000 Cost: 4.331151


## ***6. Mini Batch and Data Load***

In [59]:
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  90], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

dataset = TensorDataset(x_train, y_train)

dataloader = DataLoader(dataset, batch_size=2, shuffle=True) # 5 / 2 -> iteration = 3

model = nn.Linear(3,1)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) 

nb_epochs = 30
for epoch in range(nb_epochs + 1): # 1epoch 당
    for batch_idx, samples in enumerate(dataloader): 
#         print(batch_idx) # batch_idx : range(3)
#         print(samples) 
        x_train, y_train = samples
        # H(x) 계산
        prediction = model(x_train)

        # cost 계산
        cost = F.mse_loss(prediction, y_train)

        # cost로 H(x) 계산
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
    if epoch % 5 == 0:    
        print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, batch_idx+1, len(dataloader),
            cost.item()
            ))
        
        
# test
new_var =  torch.FloatTensor([[73, 80, 75]]) 
pred_y = model(new_var) 
print("훈련 후 입력이 73, 80, 75일 때의 예측값 :", pred_y) 
print(list(model.parameters()))

Epoch    0/30 Batch 3/3 Cost: 6049.247559
Epoch    5/30 Batch 3/3 Cost: 0.839504
Epoch   10/30 Batch 3/3 Cost: 0.721058
Epoch   15/30 Batch 3/3 Cost: 1.775649
Epoch   20/30 Batch 3/3 Cost: 0.546795
Epoch   25/30 Batch 3/3 Cost: 0.019801
Epoch   30/30 Batch 3/3 Cost: 0.191420
훈련 후 입력이 73, 80, 75일 때의 예측값 : tensor([[152.8373]], grad_fn=<AddmmBackward>)
[Parameter containing:
tensor([[1.0062, 0.7137, 0.2943]], requires_grad=True), Parameter containing:
tensor([0.2122], requires_grad=True)]


## ***7. Custom Dataset***

In [60]:
# class CustomDataset(torch.utils.data.Dataset): 
#   def __init__(self):
#   데이터셋의 전처리를 해주는 부분

#   def __len__(self):
#   데이터셋의 길이. 즉, 총 샘플의 수를 적어주는 부분

#   def __getitem__(self, idx): 
#   데이터셋에서 특정 1개의 샘플을 가져오는 함수

In [64]:
# Dataset 상속
class CustomDataset(Dataset): 
    def __init__(self):
        self.x_data = [[73, 80, 75],
                       [93, 88, 93],
                       [89, 91, 90],
                       [96, 98, 100],
                       [73, 66, 70]]
        self.y_data = [[152], [185], [180], [196], [142]]

    # 총 데이터의 개수를 리턴
    def __len__(self): 
        return len(self.x_data)

    # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
    def __getitem__(self, idx): 
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        return x, y